# Introduction
1. This system that relies on semantic similarity. It finds the text in the document that is most similar to the user's question.
2. If the user's question doesn't closely resemble the way the information is expressed in the document, the system may not find the correct answer.



## Further Development
1. Clarifying Expectation, example :
    * Chatbot: "Dana BOS digunakan untuk membiayai kegiatan operasional sekolah. Apakah Anda ingin mengetahui contoh kegiatan operasional yang dapat dibiayai oleh Dana BOS?"
2. Provide a list of example questions that the user can ask. This shows them the types of questions the chatbot is good at answering. Example:
    * Apa saja syarat pengajuan Dana BOS?
    * Bagaimana cara melaporkan penggunaan Dana BOS?
    * Sebutkan contoh kegiatan yang dapat dibiayai oleh Dana BOS.
3. Keyword Suggestions: As the user types their question, suggest relevant keywords that they can include to make their question more specific.
4. Intent Recognition (Advanced): Implement a simple intent recognition system. This would analyze the user's question and try to identify the intent behind it (e.g., "find allowed uses," "find reporting requirements"). Based on the intent, the chatbot could automatically rephrase the question to be more targeted. This requires more advanced natural language processing techniques.
5. Expand the Training Data (If Possible): If you have the ability to add more data to the system, try to find documents that explicitly list the allowed uses of Dana BOS in a clear and structured way. This will make it easier for the semantic search to find the right information.
6. Hybrid Approach (Advanced): Consider combining this semantic search approach with a more traditional keyword-based search. If the semantic search fails to find a good answer, the chatbot could fall back to a keyword search to find any relevant documents and present them to the user.

# Import Library

In [15]:
!pip install pymupdf nltk sentence-transformers langchain scikit-learn

import os
import json
import re
import nltk
import logging
import requests
import fitz
import numpy as np

from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util # Import util untuk semantic_search
from langchain.text_splitter import RecursiveCharacterTextSplitter # Import splitter baru

# Download resource NLTK
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("Downloading NLTK 'punkt'...")
    nltk.download('punkt')
    nltk.download('punkt_tab')

# Data Gathering

In [16]:
# ===============================
# 1. DATA GATHERING
# ===============================

GITHUB_RAW_URL = "https://raw.githubusercontent.com/esnanta/ai-chatbot-dana-bos-api/main/knowledge_base/"
FILES = [
    "Permendikbudriset_No_63_Tahun_2023.pdf",
]

# Direktori penyimpanan di Colab
pdf_dir = "/content/knowledge_base"
os.makedirs(pdf_dir, exist_ok=True)

# Download file PDF dari GitHub
for file in FILES:
    file_url = GITHUB_RAW_URL + file
    try:
        response = requests.get(file_url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        with open(os.path.join(pdf_dir, file), "wb") as f:
            f.write(response.content)
        print(f"✅ Berhasil mengunduh: {file}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Gagal mengunduh {file}: {e}")
    except Exception as e:
        print(f"❌ Kesalahan tak terduga saat mengunduh {file}: {e}")


# Cek file yang telah diunduh
print(f"Daftar file di {pdf_dir}: {os.listdir(pdf_dir)}")

✅ Berhasil mengunduh: Permendikbudriset_No_63_Tahun_2023.pdf
Daftar file di /content/knowledge_base: ['chunks.json', 'Permendikbudriset_No_63_Tahun_2023.pdf']


In [17]:
# ===============================
# 2. EKSTRAKSI TEKS DARI FILE PDF
# ===============================

# --- PDF Text Extraction ---
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    try:
        with fitz.open(pdf_path) as doc:  # Use context manager for safety
            text = ""
            for page in doc:
                text += page.get_text("text") + "\n"
        return text.strip()
    except Exception as e:
        raise RuntimeError(f"Gagal mengekstrak teks dari {pdf_path}: {e}")


pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]
pdf_texts = {}

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file)
    try:
        text = extract_text_from_pdf(pdf_path)
        pdf_texts[pdf_file] = text
        print(f"✅ Berhasil mengekstrak teks dari: {pdf_file}")
    except RuntimeError as re:
        print(f"❌ Kesalahan saat ekstraksi: {re}")
    except Exception as e:
        print(f"❌ Kesalahan tidak terduga pada {pdf_file}: {e}")


# Cek hasil ekstraksi
print(f"\nTotal file yang berhasil diekstrak: {len(pdf_texts)}")

✅ Berhasil mengekstrak teks dari: Permendikbudriset_No_63_Tahun_2023.pdf

Total file yang berhasil diekstrak: 1


# Preprocessing Data

In [18]:
# ===============================
# 3. PREPROCESSING TEKS
# (Fungsi clean_text Anda dipertahankan)
# ===============================
def clean_text(text):
    # This collapses multiple consecutive blank lines into a single blank line,
    # reducing unnecessary whitespace.
    text = re.sub(r'\n+', '\n', text) # Ganti \n+ -> \n dulu
    text = re.sub(r'\s+', ' ', text).strip() # Baru ganti whitespace -> spasi

    # Pasal 17. -> Pasal 17 (spasi)
    text = re.sub(r'Pasal (\d+)\.(?=\s|$)', r'Pasal \1 ', text) # Lebih robust: . diikuti spasi atau akhir string

    # Ayat (1a). -> Ayat (1a) (spasi)
    text = re.sub(r'Ayat \((\d+[a-z]?)\)\.(?=\s|$)', r'Ayat (\1) ', text)

    text = re.sub(r'http\S+|www\S+', '', text, flags=re.IGNORECASE)  # Remove URLs
    text = re.sub(r'jdih\.kemdikbud\.go\.id', '', text, flags=re.IGNORECASE)  # Remove specific website

    # Replace page number pattern '- 4 -' with '(page 4)'
    text = re.sub(r'\s-\s(\d+)\s-\s', r' (page \1) ', text) # Tetap berguna

    # Hapus pola "angka titik" di akhir kalimat yang mungkin terlewat
    # contoh: "kalimat selesai 1." -> "kalimat selesai"
    text = re.sub(r'(\w)\s+(\d+)\.\s*$', r'\1', text)

    return text

print("Cleaning texts...")
cleaned_texts = {pdf: clean_text(text) for pdf, text in pdf_texts.items()}
print(f"Cleaned {len(cleaned_texts)} documents.")

Cleaning texts...
Cleaned 1 documents.


In [19]:
# ===============================
# 4. CHUNKING TEKS (Menggunakan RecursiveCharacterTextSplitter)
# ===============================
print("Chunking texts...")

# Inisialisasi Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,      # Ukuran chunk dalam karakter (eksperimen dengan nilai ini)
    chunk_overlap=150,    # Jumlah overlap antar chunk (eksperimen)
    length_function=len,
    is_separator_regex=False,
    separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""], # Pemisah prioritas
)

# --- Fungsi tambahan untuk post-processing chunk (opsional tapi disarankan) ---
def post_process_chunk(chunk):
    """Membersihkan chunk setelah splitting."""
    # Hapus angka/bullet/simbol di awal chunk (setelah splitting mungkin muncul lagi)
    chunk = re.sub(r'^\s*[\d\-\•\*\.]+\s*', '', chunk)
    # Hapus pola nomor halaman yang mungkin masih ada di tengah
    chunk = re.sub(r'\s*\(page \d+\)\s*', ' ', chunk)
     # Hapus pola pasal/ayat yang mungkin berdiri sendiri tanpa konteks berarti setelah split
    chunk = re.sub(r'^\s*(Pasal \d+[a-z]?|Ayat \(\d+[a-z]?\))\s*$', '', chunk, flags=re.IGNORECASE)
    # Hilangkan spasi berlebihan yang mungkin muncul lagi
    chunk = re.sub(r'\s+', ' ', chunk).strip()
    return chunk

# --- Fungsi filter chunk yang terlalu pendek atau tidak relevan ---
def filter_short_or_irrelevant(chunk, min_length=50):
    """Menghapus chunk yang terlalu pendek atau hanya berisi noise."""
    if len(chunk) < min_length:
        return False
    # Contoh pola tidak relevan (bisa ditambahkan sesuai kebutuhan)
    if re.match(r'^\s*(daftar isi|kata pengantar|lampiran)\s*$', chunk, re.IGNORECASE):
        return False
    # Cek apakah chunk hanya berisi spasi atau karakter non-alfanumerik
    if not re.search(r'[a-zA-Z0-9]', chunk):
         return False
    return True

# Proses chunking untuk semua teks
all_raw_chunks = []
for pdf, text in cleaned_texts.items():
    # Gunakan text_splitter untuk memecah teks
    chunks = text_splitter.split_text(text)
    all_raw_chunks.extend(chunks)

# Post-process dan filter chunks
processed_chunks = [post_process_chunk(chunk) for chunk in all_raw_chunks]
final_chunks = [chunk for chunk in processed_chunks if filter_short_or_irrelevant(chunk)]

# Ganti nama variabel agar konsisten
all_chunks = final_chunks # final_chunks sekarang menjadi data utama kita

print(f"Total final chunks after processing and filtering: {len(all_chunks)}")
# Optional: Tampilkan beberapa contoh chunk
# for i, chunk in enumerate(all_chunks[:3]):
#      print(f"Chunk {i+1}:\n{chunk}\n-------")

Chunking texts...
Total final chunks after processing and filtering: 53


# SAVING DATA #1

* Chunk File
* Cleaned Texts File

In [20]:
# ===============================
# 5. SAVING CHUNKS
# ===============================
chunks_file = os.path.join(pdf_dir, "chunks.json")
# cleaned_texts_file = os.path.join(pdf_dir, "cleaned_texts.json") # Opsional, jika masih perlu

print(f"Saving {len(all_chunks)} final chunks...")
try:
    with open(chunks_file, "w", encoding="utf-8") as f:
        json.dump(all_chunks, f, ensure_ascii=False, indent=4)
    print(f"Final chunks saved to: {chunks_file}")
except Exception as e:
    print(f"Error saving final chunks: {e}")

# Hapus penyimpanan cleaned_texts jika tidak diperlukan lagi
# try:
#     with open(cleaned_texts_file, "w", encoding="utf-8") as f:
#         json.dump(cleaned_texts, f, ensure_ascii=False, indent=4)
#     print(f"Cleaned texts saved to: {cleaned_texts_file}")
# except Exception as e:
#     print(f"Error saving cleaned texts: {e}")

Saving 53 final chunks...
Final chunks saved to: /content/knowledge_base/chunks.json


# LOAD MODEL

In [21]:
# ===============================
# 6. MODEL EMBEDDING (Hanya SentenceTransformer)
# ===============================
print("Loading Sentence Transformer model...")
# Hapus loading CrossEncoder
# Pilih model embedder Anda
# model_name = "paraphrase-MiniLM-L3-v2" # Model yang Anda coba
model_name = "all-MiniLM-L6-v2" # Model yang lebih umum dan seringkali lebih baik (jika RAM cukup)
# model_name = "multi-qa-MiniLM-L6-cos-v1" # Alternatif bagus untuk Q&A (jika RAM cukup)
print(f"Using model: {model_name}")
print("IMPORTANT: Ensure your pre-computed embeddings (if any) were created with THIS exact model!")

try:
    embedder = SentenceTransformer(model_name)
except Exception as e:
    print(f"Error loading model {model_name}: {e}")
    # Mungkin hentikan eksekusi jika model gagal dimuat
    raise

# Dapatkan dimensi embedding dari model yang dimuat
d = embedder.get_sentence_embedding_dimension()
print(f"Embedding dimension (d): {d}")

# Encode semua chunk teks
print(f"Encoding {len(all_chunks)} chunks... (This may take a while)")
# Pastikan all_chunks tidak kosong
if not all_chunks:
    raise ValueError("Cannot encode embeddings: 'all_chunks' list is empty. Check chunking process.")

chunk_embeddings = embedder.encode(all_chunks, convert_to_numpy=True, show_progress_bar=True)
print(f"Encoded {chunk_embeddings.shape[0]} chunks.")

# HAPUS SEMUA KODE TERKAIT FAISS INDEXING (NLIST, NPROBE, quantizer, index, training, adding)

Loading Sentence Transformer model...
Using model: all-MiniLM-L6-v2
IMPORTANT: Ensure your pre-computed embeddings (if any) were created with THIS exact model!
Embedding dimension (d): 384
Encoding 53 chunks... (This may take a while)


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Encoded 53 chunks.


# SAVING DATA EMBEDDING

In [22]:
# ===============================
# 7. SAVE EMBEDDINGS (TANPA FAISS INDEX)
# ===============================
embedding_file = os.path.join(pdf_dir, "chunk_embeddings.npy")

print("Saving embeddings...")
try:
    np.save(embedding_file, chunk_embeddings)
    print(f"Embeddings saved to: {embedding_file}")
except Exception as e:
    print(f"Error saving embeddings: {e}")

Saving embeddings...
Embeddings saved to: /content/knowledge_base/chunk_embeddings.npy


# LOAD EMBEDDINGS

In [23]:
# ===============================
# 8. LOAD EMBEDDINGS (TANPA FAISS INDEX)
# ===============================
print("Loading embeddings...")
# Load embeddings dari file
if os.path.exists(embedding_file):
    # Muat embedding yang *baru saja disimpan* atau dari proses sebelumnya
    # Pastikan ini sesuai dengan model embedder yang dimuat di langkah 6!
    chunk_embeddings = np.load(embedding_file)
    print(f"Loaded embeddings with shape {chunk_embeddings.shape} from: {embedding_file}")
    # Validasi sederhana: jumlah embedding harus cocok dengan jumlah chunk
    if len(all_chunks) != chunk_embeddings.shape[0]:
         print(f"WARNING: Mismatch between loaded chunks ({len(all_chunks)}) and loaded embeddings ({chunk_embeddings.shape[0]}). Using loaded chunks count.")
         # Jika terjadi mismatch, mungkin gunakan chunk yang dimuat dari JSON saja
         try:
             with open(chunks_file, "r", encoding="utf-8") as f:
                 all_chunks = json.load(f)
             print(f"Reloaded {len(all_chunks)} chunks from {chunks_file}")
             if len(all_chunks) != chunk_embeddings.shape[0]:
                  raise ValueError(f"CRITICAL ERROR: Mismatch persists between chunks ({len(all_chunks)}) and embeddings ({chunk_embeddings.shape[0]}) even after reloading chunks.")
         except Exception as e:
             print(f"Error reloading chunks: {e}")
             raise
    # Validasi dimensi
    if chunk_embeddings.shape[1] != d:
         raise ValueError(f"CRITICAL ERROR: Loaded embedding dimension ({chunk_embeddings.shape[1]}) does not match model dimension ({d}). Ensure embeddings were generated with model '{model_name}'.")

else:
    raise FileNotFoundError(f"Embedding file {embedding_file} not found. Please run the encoding step (6) first.")

Loading embeddings...
Loaded embeddings with shape (53, 384) from: /content/knowledge_base/chunk_embeddings.npy


# Answer

In [26]:
# ===============================
# 9. FUNGSI UTAMA Q&A (Hanya Embedder + Cosine Similarity)
# ===============================

# Hapus fungsi extract_keywords dan filter_chunks_by_keywords

def answer_question(question: str,
                    chunks: list,
                    embeddings: np.ndarray,
                    embedder_model: SentenceTransformer,
                    top_n: int = 3) -> str:
    """
    Answers a question using Sentence Transformer embeddings and cosine similarity.

    Args:
        question: The user's question.
        chunks: A list of text chunks (strings).
        embeddings: A NumPy array of embeddings corresponding to the chunks.
        embedder_model: The loaded SentenceTransformer model.
        top_n: The number of top relevant chunks to return.

    Returns:
        A string containing the concatenated top_n relevant chunks.
    """
    print(f"\n🔍 Answering question: {question}")

    # 1. Encode Pertanyaan
    print("   Encoding question...")
    question_embedding = embedder_model.encode([question], convert_to_numpy=True)

    # 2. Hitung Cosine Similarity & Cari Top K
    # Menggunakan util.semantic_search yang efisien
    print(f"   Searching top {top_n} chunks using semantic search...")
    # `util.semantic_search` mengembalikan list of lists, masing-masing berisi dict
    # [{'corpus_id': index, 'score': similarity_score}, ...]
    hits = util.semantic_search(question_embedding, embeddings, top_k=top_n)

    # Ambil hasil dari query pertama (karena kita hanya encode 1 pertanyaan)
    hits = hits[0]

    # 3. Ambil Teks Chunk yang Relevan
    relevant_chunks = []
    print("   Relevant chunks found:")
    for hit in hits:
        chunk_index = hit['corpus_id']
        similarity_score = hit['score']
        if 0 <= chunk_index < len(chunks):
            print(f"     - Chunk Index: {chunk_index}, Score: {similarity_score:.4f}")
            relevant_chunks.append(chunks[chunk_index])
        else:
            print(f"     - Warning: Invalid chunk index {chunk_index} found in search results.")

    if not relevant_chunks:
        print("   -> No relevant chunks found.")
        return "Maaf, saya tidak dapat menemukan informasi yang relevan dengan pertanyaan Anda."

    # 4. Gabungkan Jawaban
    # Anda bisa memilih format penggabungan, misal <br><br> untuk HTML atau \n\n
    final_answer = "\n\n".join(relevant_chunks)
    print("   -> Final answer constructed.")
    return final_answer

def post_process_answer(answer: str) -> str:
    """
    Membersihkan dan memformat jawaban mentah agar lebih mudah dibaca.
    - Tokenisasi kalimat
    - Hapus duplikasi kalimat
    - Format sebagai bullet list
    - Filter kalimat yang sangat pendek
    """
    print("   Post-processing answer...")
    # Ganti pemisah chunk asli (\n\n) dengan spasi agar sent_tokenize bekerja lebih baik
    # pada batas antar chunk.
    text_for_tokenize = answer.replace("\n\n", " ")
    sentences = sent_tokenize(text_for_tokenize)

    # Hapus duplikasi sambil menjaga urutan (membutuhkan Python 3.7+)
    unique_sentences = list(dict.fromkeys(sentences))

    # Filter kalimat pendek dan format sebagai bullet list
    bulleted_list = []
    min_sentence_length = 15 # Tingkatkan sedikit threshold panjang minimal
    for sentence in unique_sentences:
        cleaned_sentence = sentence.strip()
        # Filter tambahan: cek jika kalimat hanya berisi spasi atau non-alpha
        if len(cleaned_sentence) > min_sentence_length and re.search(r'[a-zA-Z]', cleaned_sentence):
             # Capitalize first letter of each sentence for consistency
             formatted_sentence = cleaned_sentence[0].upper() + cleaned_sentence[1:]
             bulleted_list.append(f"* {formatted_sentence}")

    if not bulleted_list:
         print("   -> No valid sentences remained after post-processing.")
         # Kembalikan jawaban asli jika post-processing menghapus semuanya
         # atau pesan default
         return answer if answer.strip() else "Tidak ada informasi detail yang dapat ditampilkan."

    final_output = "\n".join(bulleted_list)
    print(f"   -> Post-processing complete. Generated {len(bulleted_list)} bullets.")
    return final_output

# TESTING
Chatbot menampilkan mengambil dan menampilkan 3 chunk teks yang paling mirip dengan pertanyaan pengguna sebagai jawaban.

In [27]:
# ===============================
# 10. TESTING CHATBOT
# ===============================

# Pastikan variabel yang diperlukan sudah ada dari langkah sebelumnya
if 'all_chunks' in locals() and 'chunk_embeddings' in locals() and 'embedder' in locals():

    print("\n================ TESTING CHATBOT ================")

    # Contoh pertanyaan
    test_questions = [
        "Apakah Dana BOSP dapat digunakan untuk pengembangan sumber daya manusia?",
        "Untuk apa saja Dana BOS Kinerja dapat digunakan?",
        "Kapan laporan realisasi penggunaan Dana BOSP harus disampaikan?",
        "Sebutkan komponen penggunaan dana BOS Reguler", # Contoh pertanyaan lain
        "Apa saja larangan penggunaan dana BOSP?",     # Contoh pertanyaan lain
    ]

    for question in test_questions:
        # Panggil fungsi answer_question dengan argumen yang benar
        # Hapus 'index' dan 'cross_encoder_model', tambahkan 'chunk_embeddings'
        raw_answer = answer_question(
            question=question,
            chunks=all_chunks,
            embeddings=chunk_embeddings, # Gunakan embeddings NumPy
            embedder_model=embedder,     # Berikan model embedder
            top_n=3                      # Jumlah hasil yang diinginkan
        )

        # (Opsional) Post-process jawaban (jika fungsi post_process_answer masih ada)
        # Jika Anda menghapus fungsi post_process_answer, gunakan raw_answer saja
        if 'post_process_answer' in locals():
             processed_answer = post_process_answer(raw_answer)
        else:
             processed_answer = raw_answer # Gunakan jawaban mentah jika tidak ada post-processing

        # Cetak hasil
        print(f"\n🔹 **Pertanyaan:** {question}")
        print(f"🔸 **Jawaban:**\n{processed_answer}\n")
        print("-" * 40) # Pemisah antar pertanyaan

    print("================ END OF TEST ================")

else:
    print("\nERROR: Cannot run chatbot test.")
    print("Required variables ('all_chunks', 'chunk_embeddings', 'embedder') not found.")
    print("Please ensure the loading/encoding steps (Sections 6, 8 in the modified script) have run successfully.")


================ TESTING CHATBOT ================

🔍 Answering question: Apakah Dana BOSP dapat digunakan untuk pengembangan sumber daya manusia?
   Encoding question...
   Searching top 3 chunks using semantic search...
   Relevant chunks found:
     - Chunk Index: 40, Score: 0.7352
     - Chunk Index: 35, Score: 0.7312
     - Chunk Index: 29, Score: 0.6492
   -> Final answer constructed.
   Post-processing answer...
   -> Post-processing complete. Generated 4 bullets.

🔹 **Pertanyaan:** Apakah Dana BOSP dapat digunakan untuk pengembangan sumber daya manusia?
🔸 **Jawaban:**
* Pengembangan sumber daya manusia merupakan komponen yang digunakan untuk pembiayaan dalam kegiatan penguatan sumber daya manusia dalam rangka pelaksanaan Program Sekolah Penggerak, seperti: 1) identifikasi, pemetaan potensi dan kebutuhan pelatihan; 2) penguatan pelatihan griyaan (in house training) di Satuan Pendidikan; 3) penguatan komunitas belajar bagi kepala Satuan Pendidikan dan pendidik; 4) pelatihan bersa